In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn import neighbors
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error
from sklearn.preprocessing import Imputer

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [2]:
properties = pd.read_csv('trainSetConClusters.csv')
testing = pd.read_csv('testSetConClusters.csv')

In [3]:
#properties.info()

In [4]:
#basicas
#properties = properties.loc[:, ['id', 'cluster', 'price']]
#testing = testing.loc[:, ['id', 'cluster']]

#Columnas no basicas
#'frente', 'acceso', 'agua', 'aire', 'amplio', 'balc', 'buen', 'calidad', 'caliente', 'casa', 'cochera', 'cocina', 'comedor', 'completo', 'contrafrente', 'corredor', 'credito', 'cuadra', 'cubierta', 'departamento', 'distancia_minima_privado', 'distancia_minima_publico', 'distancia_minima_subte', 'distancia_obelisco', 'doble', 'dormitorio', 'entrada', 'estrenar', 'excelente', 'expensas', 'floor', 'hall', 'lat', 'lavadero', 'living', 'lon', 'luminoso', 'madera', 'mesada', 'muebles', 'natural', 'parque', 'parrila', 'patio', 'pileta', 'placard', 'place_name_encoded', 'planta', 'principal', 'profesional', 'property_type_encoded', 'rooms', 'salida', 'seguridad', 'state_name_encoded', 'subte', 'suite', 'surface_covered_in_m2', 'surface_total_in_m2', 'terraza', 'toilette', 'vista',

#Properties y testing con id y cluster al principio, columnas no basicas, y precio al final en prop.
properties = properties.loc[:, ['id', 'cluster', 'acceso', 'agua', 'aire', 'calidad', 'caliente', 'cochera', 'cocina', 'completo', 'credito', 'cuadra', 'cubierta', 'doble', 'estrenar', 'expensas', 'floor', 'hall', 'lat', 'luminoso', 'madera', 'parque', 'patio', 'pileta', 'placard', 'planta', 'principal', 'profesional', 'property_type_encoded', 'rooms', 'salida', 'seguridad', 'suite', 'surface_total_in_m2', 'terraza', 'toilette', 'vista', 'price']]
testing = testing.loc[:, ['id', 'cluster', 'acceso', 'agua', 'aire', 'calidad', 'caliente', 'cochera', 'cocina', 'completo', 'credito', 'cuadra', 'cubierta', 'doble', 'estrenar', 'expensas', 'floor', 'hall', 'lat', 'luminoso', 'madera', 'parque', 'patio', 'pileta', 'placard', 'planta', 'principal', 'profesional', 'property_type_encoded', 'rooms', 'salida', 'seguridad', 'suite', 'surface_total_in_m2', 'terraza', 'toilette', 'vista']]

In [5]:
#Busco las correlaciones entre precio de todas las columnas
#quito todas las que tengan valor absoluto menor a 0.02

corr = properties.corr()
corr.sort_values(["price"], ascending = False, inplace = True)
print(corr.price)

price                    1.000000
suite                    0.139386
pileta                   0.130180
surface_total_in_m2      0.114009
rooms                    0.095491
toilette                 0.086850
doble                    0.079818
vista                    0.070874
principal                0.069758
seguridad                0.064425
aire                     0.061234
hall                     0.059903
planta                   0.053784
acceso                   0.052981
calidad                  0.051094
cochera                  0.050602
floor                    0.042552
terraza                  0.041996
agua                     0.041101
madera                   0.040952
salida                   0.039433
parque                   0.033168
lat                      0.031720
cubierta                 0.028999
caliente                 0.024131
cluster                  0.020791
profesional             -0.020434
estrenar                -0.022462
placard                 -0.027817
luminoso      

In [6]:
#properties['cluster'].value_counts()

In [7]:
#obtengo trainvacios y testvacios con los entries de los respectivos datasets que cumplen:
#en el train set, menos de 10 entradas por cluster
#en el test set, no tienen entradas en el train set con el mismo cluster


l = len(properties['cluster'].value_counts())

#uso un valor que se que da chico
trainvacios = properties[properties['cluster']==125]
testvacios = testing[testing['cluster']==125]

for x in range(0,l+2):
    if ((len(testing[testing['cluster']==x])==0) or (len(properties[properties['cluster']==x])<50)) and (x!=125):
    #excluyo el hardcodeado
    
        train0 = properties[properties['cluster']==x]
        trainvacios = pd.concat([trainvacios, train0])
        
        test0 = testing[testing['cluster']==x]
        testvacios = pd.concat([testvacios, test0])

In [8]:
#los datos restantes tienen más de 10 entradas en train y al menos alguna en test por barrio
#los junto en el dataframe 'trainaptos'

trainaptos= properties[~properties['id'].isin(trainvacios['id'])]
testaptos= testing[~testing['id'].isin(testvacios['id'])]
trainaptos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 85236 entries, 0 to 85648
Data columns (total 38 columns):
id                       85236 non-null object
cluster                  85236 non-null int64
acceso                   85236 non-null int64
agua                     85236 non-null int64
aire                     85236 non-null int64
calidad                  85236 non-null int64
caliente                 85236 non-null int64
cochera                  85236 non-null int64
cocina                   85236 non-null int64
completo                 85236 non-null int64
credito                  85236 non-null int64
cuadra                   85236 non-null int64
cubierta                 85236 non-null int64
doble                    85236 non-null int64
estrenar                 85236 non-null int64
expensas                 85236 non-null int64
floor                    85236 non-null float64
hall                     85236 non-null int64
lat                      85236 non-null float64
luminoso    

In [9]:
#Reviso que quedo bien el split

total = pd.concat([trainaptos, trainvacios])
print('Train:')
print('Registros:      ', len(total), ' = ', len(properties))
print('Aptos + Vacios: ', len(trainaptos), '+', len(trainvacios))

print('Clusters:       ', len(properties['cluster'].value_counts()), ' = ', len(total['cluster'].value_counts()))
print('Aptos + Vacios: ', len(trainvacios['cluster'].value_counts()), '+', len(trainaptos['cluster'].value_counts()))
print()


total = pd.concat([testaptos, testvacios])
print('Test:')
print('Registros:      ', len(total), ' = ', len(testing))
print('Aptos + Vacios: ', len(testaptos), '+', len(testvacios))

print('Clusters:       ', len(testing['cluster'].value_counts()), ' = ', len(total['cluster'].value_counts()))
print('Aptos + Vacios: ', len(testvacios['cluster'].value_counts()), '+', len(testaptos['cluster'].value_counts()))

Train:
Registros:       85649  =  85649
Aptos + Vacios:  85236 + 413
Clusters:        160  =  160
Aptos + Vacios:  48 + 112

Test:
Registros:       14166  =  14166
Aptos + Vacios:  14140 + 26
Clusters:        129  =  129
Aptos + Vacios:  17 + 112


In [10]:
#trainaptos['cluster'].value_counts()

In [11]:
#armo una funcion KNN
#devuelve un dataframe con las predicciones KNN del test y train dados y su id

def KNN(test,train,n):
    train_no_price = train.loc[:, ['acceso', 'agua', 'aire', 'calidad', 'caliente', 'cochera', 'cocina', 'completo', 'credito', 'cuadra', 'cubierta', 'doble', 'estrenar', 'expensas', 'floor', 'hall', 'lat', 'luminoso', 'madera', 'parque', 'patio', 'pileta', 'placard', 'planta', 'principal', 'profesional', 'property_type_encoded', 'rooms', 'salida', 'seguridad', 'suite', 'surface_total_in_m2', 'terraza', 'toilette', 'vista']]
     
    trainprice = train.loc[:, 'price']
    testnoids = test.loc[:, ['acceso', 'agua', 'aire', 'calidad', 'caliente', 'cochera', 'cocina', 'completo', 'credito', 'cuadra', 'cubierta', 'doble', 'estrenar', 'expensas', 'floor', 'hall', 'lat', 'luminoso', 'madera', 'parque', 'patio', 'pileta', 'placard', 'planta', 'principal', 'profesional', 'property_type_encoded', 'rooms', 'salida', 'seguridad', 'suite', 'surface_total_in_m2', 'terraza', 'toilette', 'vista']]

    knn = neighbors.KNeighborsRegressor(n, weights="uniform", algorithm="ball_tree")

    predictions = knn.fit(train_no_price, trainprice).predict(testnoids)
    
    predictions = predictions.flatten()
    ids = test.as_matrix()[:,0].flatten()
    results = pd.DataFrame()
    results['id'] = pd.Series(ids.astype('int32'))
    results['price_usd'] = pd.Series(predictions)
    
    return results

In [12]:
#defino Extreme Gradient Boosting
#tambien devuelve un dataframe con las predicciones e id
def XGB(test,train):
    #tomo id, columnas basicas, y precio para el train
    trainxgb = train.loc[:, ['id', 'acceso', 'agua', 'aire', 'calidad', 'caliente', 'cochera', 'cocina', 'completo', 'credito', 'cuadra', 'cubierta', 'doble', 'estrenar', 'expensas', 'floor', 'hall', 'lat', 'luminoso', 'madera', 'parque', 'patio', 'pileta', 'placard', 'planta', 'principal', 'profesional', 'property_type_encoded', 'rooms', 'salida', 'seguridad', 'suite', 'surface_total_in_m2', 'terraza', 'toilette', 'vista', 'price']]
    testxgb = test.loc[:, ['id', 'acceso', 'agua', 'aire', 'calidad', 'caliente', 'cochera', 'cocina', 'completo', 'credito', 'cuadra', 'cubierta', 'doble', 'estrenar', 'expensas', 'floor', 'hall', 'lat', 'luminoso', 'madera', 'parque', 'patio', 'pileta', 'placard', 'planta', 'principal', 'profesional', 'property_type_encoded', 'rooms', 'salida', 'seguridad', 'suite', 'surface_total_in_m2', 'terraza', 'toilette', 'vista']]
    longitud = len(test.columns)
    
    train = train.as_matrix()
    test = test.as_matrix()
    
    X_train = train[:,0:longitud]
    Y_train = train[:,longitud]
    X_test = test[:,0:longitud]
    
    test_ids = X_test[:,0].flatten()
    
    X_test = X_test[:,1:longitud]
    X_train = X_train[:,1:longitud]
    
    model = XGBClassifier(n_estimators=100, max_depth=3)
    model.fit(X_train, Y_train)
    
    predictions = model.predict(X_test).flatten()
    
    results = pd.DataFrame()
    results['id'] = pd.Series(test_ids.astype('int32'))
    results['price_usd'] = pd.Series(predictions)
    
    return results

In [13]:
#armo el dataframe resultados. Empieza con los test y train 'vacios'
#el resto se hará individualmente por place_name uno por uno posteriormente


#resultados = KNN(testvacios,trainvacios,25)

#Como el tamaño es chico uso XGB
resultados = XGB(testvacios, trainvacios)

resultados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 2 columns):
id           26 non-null int32
price_usd    26 non-null object
dtypes: int32(1), object(1)
memory usage: 392.0+ bytes


In [14]:
#PRUEBO XGboost:

#testx = testaptos[testaptos['cluster']==153]
#trainx = trainaptos[trainaptos['cluster']==153]

#resultados = XGB(testx,trainx)
#resultados.info()

In [15]:
#ahora saco las predicciones para los test y train aptos uno por uno
l = len(properties['cluster'].value_counts())

#samplemin define el tamaño maximo de cada set de entrenamiento
#si el set de train de uno de los cluster excede samplemin, en lugar del mismo uso un sample
samplemin= 1200

for x in range(0,l+2):
    if len(testaptos[testaptos['cluster']==x])!=0:
        testx = testaptos[testaptos['cluster']==x]
        trainx = trainaptos[trainaptos['cluster']==x]
        if len(trainaptos[trainaptos['cluster']==x])<=samplemin:
            resx = XGB(testx,trainx)
        else:
            resx = XGB(testx,trainx.sample(samplemin))
            
        resultados = pd.concat([resultados, resx])

In [16]:
#debe contener la misma cantidad que testing (14166)
resultados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14166 entries, 0 to 7
Data columns (total 2 columns):
id           14166 non-null int32
price_usd    14166 non-null object
dtypes: int32(1), object(1)
memory usage: 276.7+ KB


In [23]:
resultados.set_index(['id'], inplace=True)
resultados['id'] = resultados.index
resultados.to_csv('predicciones_cluster_XGB.csv')

In [2]:
resultados = pd.read_csv('predicciones_cluster_XGB.csv')
resultados['price'] = resultados['price_usd']
resultados = resultados.loc[:, ['id', 'price']]
resultados.sample()

,id,price
13539,2301069,125000.0


In [3]:
#resultados['id'] = resultados.index
#resultados = resultados.loc[:, ['id', 'price']]

properties = pd.read_csv('trainSetConClusters.csv')
testing = pd.read_csv('testSetConClusters.csv')

testing2 = pd.merge(testing, resultados, on='id', how='outer')
testing2.sample()

,Unnamed: 0,frente,Unnamed: 0.1,acceso,agua,aire,amplio,balc,buen,calidad,...,state_name_encoded,subte,suite,surface_covered_in_m2,surface_total_in_m2,terraza,toilette,vista,cluster,price_y
13956,13956,1,13956,0,1,0,1,1,0,0,...,4,1,0,27.0,31.0,0,0,0,51,88000.0


In [4]:
testing2['price'] = testing2['price_y']
testing2 = testing2.drop('price_y', 1)
testing2 = testing2.drop('price_x', 1)
testing2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14166 entries, 0 to 14165
Data columns (total 69 columns):
Unnamed: 0                  14166 non-null int64
 frente                     14166 non-null int64
Unnamed: 0.1                14166 non-null int64
acceso                      14166 non-null int64
agua                        14166 non-null int64
aire                        14166 non-null int64
amplio                      14166 non-null int64
balc                        14166 non-null int64
buen                        14166 non-null int64
calidad                     14166 non-null int64
caliente                    14166 non-null int64
casa                        14166 non-null int64
cochera                     14166 non-null int64
cocina                      14166 non-null int64
comedor                     14166 non-null int64
completo                    14166 non-null int64
contrafrente                14166 non-null int64
corredor                    14166 non-null int64
created_o

In [5]:
total = pd.concat([properties, testing2])

In [6]:
total.sample(5)

,frente,Unnamed: 0,Unnamed: 0.1,acceso,agua,aire,amplio,balc,buen,calidad,...,salida,seguridad,state_name_encoded,subte,suite,surface_covered_in_m2,surface_total_in_m2,terraza,toilette,vista
23742,0,23742,50022,1,0,1,1,1,1,1,...,1,0,1,0,1,560.0,2674.0,1,1,1
48862,0,48862,95564,0,0,0,0,0,0,0,...,0,0,1,0,0,140.0,300.0,0,0,0
596,1,596,596,0,0,0,0,1,0,0,...,0,0,4,0,0,38.0,44.0,0,0,0
41098,0,41098,81986,0,0,0,0,1,0,0,...,0,0,0,0,0,110.0,200.0,1,0,0
58095,1,58095,135445,0,1,1,1,1,0,1,...,0,1,4,1,0,34.0,36.0,0,0,0


In [7]:
#total['precio_por_cluster']= total.groupby('price')['price'].transform('mean')
total['precio_por_cluster']= total['price'].groupby(total['cluster']).transform('mean')
total.sample(5)

,frente,Unnamed: 0,Unnamed: 0.1,acceso,agua,aire,amplio,balc,buen,calidad,...,seguridad,state_name_encoded,subte,suite,surface_covered_in_m2,surface_total_in_m2,terraza,toilette,vista,precio_por_cluster
76507,1,76507,178067,0,0,0,1,1,0,0,...,0,4,0,0,135.0,86.0,1,1,1,529814.841035
24492,0,24492,51206,0,0,0,1,1,0,1,...,0,4,0,0,85.0,96.0,1,0,0,240128.443108
9984,1,9984,9984,0,1,1,1,1,0,0,...,0,4,0,0,39.0,48.0,0,0,1,173204.794328
49515,1,49515,97140,0,0,0,0,0,0,0,...,0,0,0,0,548.0,548.0,1,0,0,895678.770053
1585,0,1585,2831,0,0,0,0,0,0,0,...,0,4,0,0,57.0,57.0,0,0,0,173204.794328


In [9]:
test= total[~total['id'].isin(properties['id'])]
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14166 entries, 0 to 14165
Data columns (total 70 columns):
 frente                     14166 non-null int64
Unnamed: 0                  14166 non-null int64
Unnamed: 0.1                14166 non-null int64
acceso                      14166 non-null int64
agua                        14166 non-null int64
aire                        14166 non-null int64
amplio                      14166 non-null int64
balc                        14166 non-null int64
buen                        14166 non-null int64
calidad                     14166 non-null int64
caliente                    14166 non-null int64
casa                        14166 non-null int64
cluster                     14166 non-null int64
cochera                     14166 non-null int64
cocina                      14166 non-null int64
comedor                     14166 non-null int64
completo                    14166 non-null int64
contrafrente                14166 non-null int64
corredor 

In [10]:
train= total[~total['id'].isin(test['id'])]
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 85649 entries, 0 to 85648
Data columns (total 70 columns):
 frente                     85649 non-null int64
Unnamed: 0                  85649 non-null int64
Unnamed: 0.1                85649 non-null int64
acceso                      85649 non-null int64
agua                        85649 non-null int64
aire                        85649 non-null int64
amplio                      85649 non-null int64
balc                        85649 non-null int64
buen                        85649 non-null int64
calidad                     85649 non-null int64
caliente                    85649 non-null int64
casa                        85649 non-null int64
cluster                     85649 non-null int64
cochera                     85649 non-null int64
cocina                      85649 non-null int64
comedor                     85649 non-null int64
completo                    85649 non-null int64
contrafrente                85649 non-null int64
corredor 

In [11]:
train.to_csv('train_precioxcluster.csv')
test.to_csv('test_precioxcluster.csv')